In [1]:
import pandas as pd
import numpy as np
import re
import string 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
data_fake=pd.read_csv('fake.csv')
data_true=pd.read_csv('true.csv')

In [3]:
data_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [4]:
data_fake['class']=0
data_true['class']=1

In [5]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [6]:
fake_manual=data_fake.tail(10)
true_manual=data_true.tail(10)
testing_manual=pd.concat([fake_manual,true_manual])

In [7]:
testing_manual.to_csv('manual_testing.csv')

In [8]:
df_merge=pd.concat([data_fake,data_true])
df_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [9]:
df=df_merge.drop(['title','subject','date'], axis=1)
df

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",1
21414,MINSK (Reuters) - In the shadow of disused Sov...,1
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [10]:
df=df.sample(frac=1)
df

,text,class
9359,CNN political analyst just spent her Thanksgiv...,0
1825,BOSTON/WASHINGTON (Reuters) - A newly fraying ...,1
12374,CLINTON ADMITS SHE CAN T HANDLE THE LOSS CAN...,0
8116,PARIS (Reuters) - Edinson Cavani silenced his ...,1
14313,New York City that great melting pot of divers...,0
...,...,...
13849,Vietnam has supposedly made modest gains in re...,0
12297,LONDON (Reuters) - Britain has expressed grave...,1
19201,DUBAI (Reuters) - A solution in Yemen s two an...,1
8681,The film industry wants us to believe that it ...,0


In [11]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [12]:
def word_drop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [13]:
df['text']=df['text'].apply(word_drop)

In [14]:
x=df['text'].values
y=df['class'].values

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=.20)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [17]:
vectorization=TfidfVectorizer()
xv_train=vectorization.fit_transform(x_train)
xv_test=vectorization.transform(x_test)

In [18]:
#### logistic regresion


In [19]:
from sklearn.linear_model import LogisticRegression
regression=LogisticRegression()
regression.fit(xv_train,y_train)

LogisticRegression()

In [20]:
regression.score(xv_test, y_test)

0.9864142538975501

In [21]:
pred=regression.predict(xv_test)

In [22]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4620
           1       0.99      0.99      0.99      4360

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



#### decision tree

In [23]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(xv_train,y_train)

DecisionTreeClassifier()

In [24]:
pred_dt=dt.predict(xv_test)

In [25]:
dt.score(xv_test,y_test)

0.9953229398663697

In [26]:
print(classification_report(y_test,pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4620
           1       1.00      0.99      1.00      4360

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [27]:
## gradient boosting classifier

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
gbc.fit(xv_train,y_train)

GradientBoostingClassifier()

In [29]:
pred_gbc=gbc.predict(xv_test)

In [30]:
gbc.score(xv_test,y_test)

0.9944320712694877

In [31]:
print(classification_report(y_test,pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4620
           1       0.99      1.00      0.99      4360

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [32]:
### random forest classifier

In [33]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(xv_train,y_train)

RandomForestClassifier()

In [34]:
pred_rf=rf.predict(xv_test)

In [35]:
rf.score(xv_test,y_test)

0.988641425389755

In [36]:
print(classification_report(y_test,y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4620
           1       1.00      1.00      1.00      4360

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [37]:
def output_lable(n):
    if n==0:
        return 'fake news'
    elif n==1:
        return 'true news'

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_drop) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = regression.predict(new_xv_test)
    pred_DT = dt.predict(new_xv_test)
    pred_GBC = gbc.predict(new_xv_test)
    pred_RFC = rf.predict(new_xv_test)
    
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]), 
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))                                      
                                                                                                             

In [ ]:
news=str(input())
manual_testing(news)